In [1]:
# Import needed libraries
import numpy as np
from scipy.integrate import solve_bvp
import matplotlib.pyplot as plt
from integral_constraints import C, C_dx, C_dy, K, K_dvx, K_dvy, K_ddelta


# 1. Introduction

### Abstract

Efficient pathfinding is a critical aspect of competitive racing across various disciplines, including marathons, cycling tours, and Formula 1 events. This paper delves into the pursuit of identifying optimal trajectories for completing laps on NASCAR tracks with the primary objective of minimizing lap times. A key consideration in achieving high speeds on such tracks is the ability to navigate turns effectively, as exceeding the optimal speed threshold can compromise traction and stability, leading to undesirable outcomes such as skidding or flipping. The concept of the racing line, which involves maximizing turn radii by starting on the outside, hitting the apex, and finishing on the outside of a turn, plays a pivotal role in this endeavor.

Through a comprehensive analysis, this study integrates principles of optimal control, vehicle steering dynamics, and friction constraints to formulate strategies for determining the fastest path around a NASCAR track. By leveraging these methodologies, we aim to enhance our understanding of the complex interplay between driving dynamics and track geometry, ultimately contributing to the advancement of racing performance and strategy. The findings presented herein offer insights into the intricacies of path optimization in high-speed racing environments and lay the groundwork for further research in this domain.

# 2. Background 

### Problem Description

At first glance, it may seem straightforward to determine the best path for a racecar around an F-1 track by solely controlling its acceleration in the x and y directions. However, this approach overlooks important real-world factors.

In reality, steering dynamics and the limited traction available to prevent sliding or flipping are crucial considerations in finding the optimal path. To accurately represent this, we must incorporate these physical constraints, including introducing another control for steering wheel turn acceleration, which must be managed to maintain sufficient traction.

In our study, we compared two methods: one using a naive approach and the other considering traction constraints. Through this comparison, we aimed to illustrate how the car's navigation around the track differs under these conditions. This highlights the significance of addressing real-world complexities when optimizing racing paths.



### Relevant Research

Many have formed mathematical models for the motion of automobiles in a variety of settings including optimal control. A common choice for modeling the motion of an automobile is the Kinematic Bicycle Model which simplifies the dynamics of an automobile. Victor Freire and Xiangru Xu, for example, used the Kinematic Bicycle Model when solving an optimal control problem for maximizing the safety of automobile trips [1]. In addition, Limebeer, Bastin, Warren, & Fensham employed the Kinematic Bicycle Model in their optimal control problem for NASCAR race cars [2]. In accordance with this research, we also use the Kinematic Bicycle Model to simplify the control problem without deviating too far from real world kinematics. We outline the application of the Kinematic Bicycle Model to our optimal control problem below. 

# 3. Mathematical Representation

### The Setup

We first defined the variables for the representation of a car going around an F-1 racetrack as:

\begin{align*}
x &= \text{x position} \\
y &= \text{y position} \\
v_x &= \text{x velocity} \\
v_y &= \text{y velocity} \\
\theta &= \text{vehicle orientation with respect to the x-axis} \\
\delta &= \text{angle of front wheels with respect to the vehicle's orientation} \\
L &= \text{Length of car measured from front wheel axis to back wheel axis} \\
M &= \text{maximum allowable centripetal acceleration} \\
u_{\text{acc}} &= \text{acceleration} \\
u_{\text{angle}} &= \text{change in steering angle}
\end{align*}

where $u_{acc}$ and $u{angle}$ are our control variables.

We defined our state equations as follows:

$\begin{bmatrix}
\dot{x} \\
\dot{y} \\
\dot{v}_x \\
\dot{v}_y \\
\dot{\theta} \\
\dot{\delta} \\
\end{bmatrix} = \begin{bmatrix}
    v_x \\
    v_y \\
    u_{acc} \cdot \cos(\theta) \\
    u_{acc} \cdot \sin(\theta) \\
    \sqrt{{v_x}^2 + {v_y}^2} \cdot \frac{\tan(\delta)}{L} \\
    u_{angle}
\end{bmatrix}$


With an unmodified cost functional of 

$J[u] = \int\limits_0^{t_f}dt$ 


Using the fact that centripetal acceleration is $\frac{v^2}{r}$ where $r$ is the turn radius which is dependent on $L$, we arive at our centripetal acceleration constraint:

$\frac{{v_x}^2 + {v_y}^2}{L}\tan(\delta) - M < 0$

And finally we have the constraints of the race track:

FIXME - add in the actual constraints of the race track

















# 4. Derivation of Solution

### Deriving the Modified Cost Functional

We enforce the constraints above by defining functions with extremely high penalties for violating the constraints and including these functions in the cost functional. We first define the function that describes the topology of the racetrack, penalizing heavily for movement inside and outside the track:

$C(x,y) = \frac{3}{(\frac{x^2}{15} + \frac{y^2}{10})^{20} + 1}
-\frac{3}{(\frac{x^2}{10} + \frac{y^2}{5})^{70} + 1}
+\frac{3}{((x-1)^2 + y^2)^{20} + 1}
+\frac{3}{((x+1)^2 + y^2)^{20} + 1}
+\frac{3}{(x^2 + y^2)^{20} + 1}$

Next, we define the function that gives a severe penalty for violating the centripetal acceleration constraint:

$K(v_x, v_y, \delta) = \left(\frac{1}{\frac{{v_x}^2 + {v_y}^2}{L}\tan(\delta) - M} \right)^{70}$

We then define a function that penalizes unreasonably high acceleration values:

$W(u_{acc}, u_{angle}) = w_1{u_{acc}}^2 + w_2{u_{angle}}^2$

Finally, we incorporate all of these penalties into our new cost functional:

$J[u] = \int\limits_0^{t_f} 1 + C(x,y) + K(v_x, v_y, \delta) + W(u_{acc}, u_{angle})dt$ 



### The Hamiltonian, Co-State Equations, and Boundary Equations (Whoever made the Latex for the poster put it here as well)


Starting with $H = p \cdot f - L$, our full hamiltonian for this problem is:

$H = p_1v_x + p_2v_y + p_3u_{acc}\cos(\theta) + p_4u_{acc}\sin(\theta) + p_5\sqrt{v_x^2 + v_y^2}\frac{\tan(\delta)}{L} + p_6u_{angle} - (1 + C(x,y) + K(v_x, v_y, \delta) + W(u_{acc}, u_{angle}))$

Using this Hamiltonian and applying Pontryagin's Maximum Principle, we obtain the costate equations:

$\begin{bmatrix}
\dot{p_1} \\
\dot{p_2} \\
\dot{p_3} \\
\dot{p_4} \\
\dot{p_5} \\
\dot{p_6} \\
\end{bmatrix} =
\begin{bmatrix}
C_x \\
C_y \\
-p_1 - p_5\frac{v_x}{\sqrt{v_x^2 + v_y^2}}\frac{\tan(\delta)}{L} + K_{v_x}\\
-p_2 - p_5\frac{v_y}{\sqrt{v_x^2 + v_y^2}}\frac{\tan(\delta)}{L} + K_{v_y}\\
p_3u_{acc}\sin(\theta) - p_4u_{acc}\cos(\theta) \\
-\frac{p_5}{L}\sqrt{v_x^2 + v_y^2}\sec^{2}(\delta) + K_{\delta} \\
\end{bmatrix}$

# 5. Interpretations (they want a section called that)

### Naive Approach

In [ ]:
def naive_system():
    W1 = 100
    W2 = .02
    n = 200
    t = np.linspace(0, 1, n)

    C0 = C(1.25, 1.25, W1)

    def ode(t, s, p):
        return p[0] * np.array([
            s[2],
            s[3],
            1/(2*W2)*s[6],
            1/(2*W2)*s[7],
            C_dx(s[0], s[1], W1),
            C_dy(s[0], s[1], W1),
            -s[4],
            -s[5]
        ])
    
    def bc(ya, yb, p):
        return np.array([
            ya[0] + 1, ya[1] + 1.1, ya[2], ya[3], yb[0] - 1, yb[1] - 1.1, yb[2], yb[3],
            
            # H(tf) = 0
            yb[4]*yb[2] + yb[5]*yb[3] + yb[6]**2/(2*W2) + yb[7]**2/(2*W2) - (1 + C0 + (yb[6]**2+yb[7]**2/(4*W2)))
        ])
    
    guess = np.ones((8, n))

    path_x = np.concatenate((np.linspace(-1.1, 2.1, 100), 2.1*np.ones(75), np.linspace(1.1, 2.1, 25)[::-1]))
    path_y = np.concatenate((-1.1*np.ones(100), np.linspace(-1.1,1.1,75), 1.1*np.ones(25)))


    guess[0] = path_x
    guess[1] = path_y
    p0 = np.array([.5])

    sol = solve_bvp(ode, bc, t, guess, p0, max_nodes=30000)

    X = np.linspace(-4, 4, 200)
    Y = np.linspace(-3, 3, 200)
    X, Y = np.meshgrid(X, Y)
    Z = C(X, Y)


    plt.plot(guess[0], guess[1], "--k", label="Inital Guess")
    plt.contour(X, Y, Z)
    plt.plot(sol.sol(t)[0], sol.sol(t)[1], "tab:orange", label="Path of Car")
    plt.scatter(-1, -1.1, marker="*", color="red", label="Start")
    plt.scatter(1, 1.1, marker="*", color="green", label="End")
    plt.xlim([-4,4])
    plt.ylim([-3,3])
    plt.legend()
    plt.title(f"{sol.p[0]:.2F} seconds")
    plt.show()

    return sol.p[0]

naive_system()


### Graphs

### (i) Initial Guess of All Ones

![Image Title](/Users/user/Vol4Project-1/figures/no_guess.png)

### (ii) Initial Guess of Going Straight Through the Track

![Image Title](/Users/user/Vol4Project-1/figures/bad_guess.png)

### (iii) Initial Guess That Avoids Edges of The Track

![Image Title](/Users/user/Vol4Project-1/figures/better_guess.png)

### Approach with Centripetal Acceleration Constraint

In [ ]:
# Put Ilha's code here

### Graphs

### Comparisons with other non-optimal solutions?

# 6. Appendix

[1] V. Freire and X. Xu, "Optimal Control for Kinematic Bicycle Model With Continuous-Time Safety Guarantees: A Sequential Second-Order Cone Programming Approach," in IEEE Robotics and Automation Letters, vol. 7, no. 4, pp. 11681-11688, Oct. 2022

[2] Limebeer, D.J.N., Bastin, M., Warren, E., & Fensham, H. G. (2023). Optimal control of a NASCAR - specification race car. Vehicle System Dynamics, 61(5), 1210-1235
